In [36]:
import time
import mlrose_hiive as mlrh
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [7, 7]

random_state = 923
np.random.seed(923)

In [37]:
def plot_fitness_iteration(curve, title, x="Iteration", y="Fitness"):
    plt.xlabel(x)
    plt.ylabel(y)
    plt.title(title)
    plt.plot(range(len(curve)), curve, label=y, lw=2)
    plt.legend(loc='upper left')
    plt.show()

# N Queens Problem

In [46]:
# Problem definition
length = 32
global eval_count
eval_count = 0
# Define alternative N-Queens fitness function for maximization problem
def queens_max(state):
    global eval_count
    eval_count += 1
    # Initialize counter
    fitness_cnt = 0

    # For all pairs of queens
    for i in range(len(state) - 1):
        for j in range(i + 1, len(state)):
            # Check for horizontal, diagonal-up and diagonal-down attacks
            if (state[j] != state[i]) \
                and (state[j] != state[i] + (j - i)) \
                and (state[j] != state[i] - (j - i)):

                # If no attacks, then increment counter
                fitness_cnt += 1
    return fitness_cnt
# Initialize custom fitness function object
q_fitness = mlrh.CustomFitness(queens_max)
# q_fitness = mlrose.Queens()
problem = mlrh.QueensOpt(length=length, fitness_fn=q_fitness, maximize=True)
experiment_name = "queen"
output_directory = "queen"

## MIMIC

In [51]:
mim = mlrh.MIMICRunner(problem=problem,
                       experiment_name=experiment_name,
                       output_directory=output_directory,
                       seed=random_state,
                       population_sizes=[50, 100, 200, 500, 1000, 2000],
                       keep_percent_list=[0.1, 0.25, 0.5, 0.7, 0.9],
                       iteration_list=[50],
                       use_fast_mimic=True)
mim_stats, mim_curve = mim.run()
show=10
columns = ['Time', 'Fitness', 'Population Size', 'Keep Percent']
pd.set_option('display.max_rows',show)
df=pd.read_csv("./queen/queen/mimic__queen__run_stats_df.csv")
df[columns].sort_values(by=['Fitness','Population Size'], ascending=(False,True)).head(show)

,Time,Fitness,Population Size,Keep Percent
53,272.345509,621.0,2000,0.25
55,244.776806,620.0,2000,0.50
51,323.641323,619.0,2000,0.10
33,72.889820,618.0,500,0.25
43,141.159975,618.0,1000,0.25
45,131.189662,618.0,1000,0.50
25,37.519630,617.0,200,0.50
35,70.794388,617.0,500,0.50
37,74.096312,617.0,500,0.70
57,272.465629,617.0,2000,0.70


## Genetic Algorithm

In [52]:
ga = mlrh.GARunner(problem=problem,
                   experiment_name=experiment_name,
                   output_directory=output_directory,
                   seed=random_state,
                   max_attempts=20,
                   population_sizes=[50, 100, 200, 500, 1000, 2000],
                   mutation_rates=[0.1, 0.25, 0.5, 0.7, 0.9],
                   iteration_list=[50])
ga_stats, ga_curve = ga.run()

In [53]:
show=10
columns = ['Time', 'Fitness', 'Population Size', 'Mutation Rate']
pd.set_option('display.max_rows',show)
df=pd.read_csv("./queen/queen/ga__queen__run_stats_df.csv")
df[columns].sort_values(by=['Fitness','Mutation Rate'], ascending=(False,True)).head(show)

,Time,Fitness,Population Size,Mutation Rate
13,4.249922,620.0,100,0.25
5,2.119506,618.0,50,0.50
1,2.082971,617.0,50,0.10
15,4.198301,617.0,100,0.50
9,2.179527,617.0,50,0.90
11,4.268715,616.0,100,0.10
3,2.030952,616.0,50,0.25
31,20.392658,614.0,500,0.10
51,82.420702,614.0,2000,0.10
23,8.086936,614.0,200,0.25


In [8]:
## OLD ##
show=10
columns = ['Time', 'Fitness', 'Population Size', 'Mutation Rate']
pd.set_option('display.max_rows',show)
df=pd.read_csv("./queen/queen/ga__queen__run_stats_df.csv")
df[columns].sort_values(by=['Fitness'], ascending=False).head(show)

,Time,Fitness,Population Size,Mutation Rate
5,7.596768,591.0,100,0.50
3,7.136895,590.0,100,0.25
7,7.620422,590.0,100,0.75
1,7.482664,588.0,100,0.10
9,6.443608,583.0,100,0.90
21,34.787708,579.0,1000,0.10
15,14.171610,578.0,500,0.50
11,14.838323,578.0,500,0.10
27,23.551499,578.0,1000,0.75
23,20.079519,578.0,1000,0.25


## Simulated Annealing

In [59]:
sa = mlrh.SARunner(problem=problem,
                   experiment_name=experiment_name,
                   output_directory=output_directory,
                   seed=random_state,
                   max_attempts=200,
                   iteration_list=[1000],
                   temperature_list=[0.05, 0.1, 0.5, 1, 10, 30],
                   decay_list=[mlrh.GeomDecay, mlrh.ExpDecay, mlrh.ArithDecay])
sa_stats, sa_curve = sa.run()

show=10
columns = ['Time', 'Fitness', 'Temperature', 'schedule_type']
pd.set_option('display.max_rows',show)
df=pd.read_csv("./queen/queen/sa__queen__run_stats_df.csv")
df[columns].sort_values(by=['Fitness','Temperature'], ascending=(False,True)).head(show)

,Time,Fitness,Temperature,schedule_type
13,1.041637,629.0,0.50,geometric
15,1.018552,629.0,0.50,exponential
1,1.022927,627.0,0.05,geometric
3,1.028348,627.0,0.05,exponential
5,1.008428,627.0,0.05,arithmetic
7,1.021032,627.0,0.10,geometric
9,1.011133,627.0,0.10,exponential
11,1.020333,627.0,0.10,arithmetic
25,1.160197,627.0,10.00,geometric
19,1.053109,626.0,1.00,geometric


In [55]:
#[100,500]

,Time,Fitness,Temperature,schedule_type
2,0.547717,625.0,0.05,geometric
5,0.541457,625.0,0.05,exponential
8,0.525250,625.0,0.05,arithmetic
11,0.548398,625.0,0.10,geometric
14,0.572367,625.0,0.10,exponential
17,0.546381,625.0,0.10,arithmetic
20,0.557766,625.0,0.50,geometric
23,0.563811,625.0,0.50,exponential
32,0.602143,624.0,1.00,exponential
29,0.583631,623.0,1.00,geometric


# Randomized Hill Climbing

In [66]:
rhc = mlrh.RHCRunner(problem=problem,
                    experiment_name=experiment_name,
                    output_directory=output_directory,
                    seed=random_state,
                    max_attempts=100,
                    iteration_list=[1000],
                    restart_list=[5,10])
rhc_stats, rhc_curve = rhc.run()
show=10
columns = ['Time', 'Fitness', 'Restarts', 'current_restart']
pd.set_option('display.max_rows',show)
df=pd.read_csv("./queen/queen/rhc__queen__run_stats_df.csv")
df[columns].sort_values(by=['Fitness','Restarts'], ascending=(False,True)).head(show)

,Time,Fitness,Restarts,current_restart
3,0.976699,626.0,5,1
15,1.008304,626.0,10,1
1,0.344089,623.0,5,0
11,2.872223,623.0,5,5
13,0.350676,623.0,10,0
23,2.959783,623.0,10,5
7,1.820519,622.0,5,3
19,1.869516,622.0,10,3
25,3.600771,622.0,10,6
31,5.460560,622.0,10,9


In [60]:
show=10
columns = ['Time', 'Fitness', 'Restarts', 'current_restart']
pd.set_option('display.max_rows',show)
df=pd.read_csv("./queen/queen/rhc__queen__run_stats_df.csv")
df[columns].sort_values(by=['Fitness','Restarts'], ascending=(False,True)).head(show)

,Time,Fitness,Restarts,current_restart
62,11.691179,626.0,10,9
63,11.691179,626.0,10,9
50,7.040023,625.0,10,6
51,7.040023,625.0,10,6
2,0.627853,624.0,5,0
3,0.627853,624.0,5,0
10,2.135167,624.0,5,2
11,2.135167,624.0,5,2
14,3.314289,624.0,5,3
15,3.314289,624.0,5,3
